# Inteiros e números decimais (Float)

Bem aqui lembramos , por exemplo, que números representados no nosso código são *numeric literals*: 1 e 1.0 são numeric literals.
Mas as representações como objetos desses mesmos números na memória, são chamados de *numeric primitives*.

Aqui temos as primitivas numéricas (prefiro *numeric primitives*) presentes em Julia:
* Inteiros
    * Int8, Int16, Int32, Int64, Int128
    * UInt8, UInt16, UInt32, UInt64, UInt128
    * Bool

* Decimais
    * Float16 (half precision), Float32(single precision), Float64(double precision)
    

Lembrando que U vem de *<u>unsigned</u>*, o que significa que são primitivas onde só permitem números sem sinal, ou seja, ***números positivos***.

 O `typeof` diz-nos o tipo de uma variável/constante/valor, e caso este seja inteiro, a primitiva pode mudar consoante a arquitetura do vosso computador (32 ou 64 bits).

In [75]:
# No meu caso dará Int64, pois tenho um CPU de 64 bits
typeof(1)

Int64

Pode-se obter a informação do número de bits usados pelo CPU, utilizando o `Sys.WORD_SIZE`

In [76]:
Sys.WORD_SIZE

64

Também se pode usar os *types* `UInt` e `Int` assim como as restantes primitivas citadas acima. Eles podem inclusive, modificar o tipo das variáveis

In [77]:
a = UInt8(245)
println( typeof(a) )

a = UInt(a)
println( typeof(a) )

UInt8
UInt64


***Nota***: Inteiros que não possa ser representados em 32 bits, são convertidos para 64 bits, independemente da arquitetura.


Algo também bastante interessante é que números em representação hexadecimal, octal e binário são sempre representados por primitivas *unsigned*, isto deve-se ao facto de que números negativos são convertidos para uma representação binária [Complemento para dois](https://pt.wikipedia.org/wiki/Complemento_para_dois).

A quantidade de números 1 à esquerda, dita o tamanho mínimo necessário para armazenar o número. 
Por exemplo 0xff pode ser armazenado em `UInt8`, porém 0xfff já será armazenado em `UInt16`.

In [78]:
println("1: ", typeof( 1 ) )
println("-1: ", typeof( -1 ) )
println("0x1: ", typeof( 0x1 ) )
println("-0x1: ", typeof( -0xfff ) )
-0x1

1: Int64
-1: Int64
0x1: UInt8
-0x1: UInt16


0xff

Caso queiramos saber o número mínimo e máximo que uma primitiva pode representar, basta-nos usar as funções `typemin` e `typemax`

In [79]:
typemin(Int16)

-32768

In [80]:
typemax(Int16)

32767

## Overflow

Quando tentamos fazer um overflow (armazenar um valor tão grande/pequeno que nenhuma primitiva consegue representar), Julia vai aplicar [aritmética modular](https://en.wikipedia.org/wiki/Modular_arithmetic) fazendo com que os valor volte para o intervalo da primitiva.

Caso queiramos pode representar esse número sem usar este 'wraparound', devemos fazer uso do `BigInt`

In [81]:
# Integer Overflow (64bits)
x = typemax(Int64)

9223372036854775807

In [82]:
# Vai transformá-lo no extremo negativo
x + 1 == typemin(Int64)

true

In [83]:
# Contornar esse efeito com o uso do BigInt
x = BigInt(typemax(Int64))
x + 1

9223372036854775808

## Divisões

As divisões também são bem interessantes por aqui. Todos sabemos que não podemos dividir por 0 e se usarmos a função `div` ela retornará um erro de divisão, assim como a divisão do menor número possível por -1, como podemos ver abaixo.

In [84]:
# Dividir 1 por 0
div(1, 0)

LoadError: DivideError: integer division error

In [85]:
# Dividir o typemin de u por -1
div(typemin(Int64), -1)

LoadError: DivideError: integer division error

Porém se dividirmos um número por 0 sem o uso da função `div`, vamos obter um valor infinito (`Inf`) do tipo `Float`.

Para a divisão do menor número negativo por -1 dará um valor próximo do maior número possível.

In [86]:
println("DIVIDIR NÚMERO POR 0")
println( 1/0 )
println( typeof( 1/0 ) )
println("\nDIVIDIR MENOR NÚMERO NEGATIVO POR -1")
println( typemin(Int64)/-1 ≈ typemax(Int64))
println( typeof( typemin(Int64)/-1 ) )

DIVIDIR NÚMERO POR 0
Inf
Float64

DIVIDIR MENOR NÚMERO NEGATIVO POR -1
true
Float64


Também temos as funções `rem` (resto) e `mod` (resto).

A diferença entre elas confude-me um pouco, sei que `rem` é o mesmo que `x % y`, onde o sinal do divisor (y, neste caso) não importa. Quando o dividendo e divisor têm o mesmo sinal, `rem` e `mod` dão o mesmo resultado.

Muitas coisas em Julia são iguais ao MATLAB e essas funções confirmam isso! Ao pesquisar um pouco, encontrei uma resposta que ajudou-me a elucidar um pouco.

Pensemos então na divisão de $\frac{5}{-3}$

<hr>


***Caso do `rem`***

Este é o resto normal como se fizéssemos a conta manualmente, ou seja, o resto será ***2***.

<hr>

***Caso do `mod`***
Este é um caso que está relacionado à [aritmética modular](https://en.wikipedia.org/wiki/Modular_arithmetic).

Pensemos nos múltiplos do nosso divisor, que no caso é 3 => $-3\mathbb{Z} = -3\{...,-3,-2,-1,0,1,2,3,...\} = \{...,9,6,3,0,-3,-6,-9,...\}$

O número que mais se proxima de 5 é o 6, logo temos de somar ***-1*** e esse será o nosso resto.

<hr>

A resposta completa está aqui (em inglês): [rem & mod](https://www.quora.com/I-am-a-beginner-in-MATLAB-programming-What-is-the-difference-between-the-two-operations-rem-and-mod-How-are-both-functionally-different)

<hr>

****Regra Geral****:

* Dividendo e divisor têm o mesmo sinal? Se sim, ambas vão retornar o mesmo.
* Para o `rem` só o sinal do dividendo muda o resultado final.